# Лабораторна робота No2
### Наука про дані: підготовчий етап
#### Мета роботи:
ознайомитися з основними кроками по роботі з даними – workflow
від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі
та природу даних, над якими виконується аналітичні операції
Основні поняття: сирі дані (raw data), підготовка даних (data preparation)

Виконав *ФБ-35 Шаєв Максим*

# Хід виконання роботи
1. Створив віртуальне середовище ds1_venv, та завантажив необхідні бібліотеки
   `python -m venv .ds1_venv`

2. Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При
зберіганні файлу до його імені потрібно додати дату та час завантаження.
Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

Імпортуємо необхнідні бібліотеки

In [1]:
import pandas as pd
import urllib.request
from datetime import datetime
import os
print("Setup Complete")

Setup Complete


Основний код:

In [2]:
def csv_down(index):
    dir_files = os.listdir('C:/Users/ender/PycharmProjects/pythonProject/DataScienceAssigments/csv_prov')
    try:
        for x in dir_files:
            if int(x.split('_')[2]) == index:
                print(f'-File {index} already downloaded')
                return
    except IndexError:
        print('Bad name for file')
    url=f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={index}&year1=1981&year2=2024&type=Mean"
    vhi_url = urllib.request.urlopen(url)
    now = datetime.now()
    date_and_time = now.strftime("%d%m%Y%H%M%S")
    out = open(f'C:/Users/ender/PycharmProjects/pythonProject/DataScienceAssigments/csv_prov/vhi_id_{index}_{date_and_time}.csv', 'wb')
    out.write(vhi_url.read())
    out.close()
    print(f'+File {index} downloaded')


for i in range(1,26):
    csv_down(i)

-File 1 already downloaded
-File 2 already downloaded
-File 3 already downloaded
-File 4 already downloaded
-File 5 already downloaded
-File 6 already downloaded
-File 7 already downloaded
-File 8 already downloaded
-File 9 already downloaded
-File 10 already downloaded
-File 11 already downloaded
-File 12 already downloaded
-File 13 already downloaded
-File 14 already downloaded
-File 15 already downloaded
-File 16 already downloaded
-File 17 already downloaded
-File 18 already downloaded
-File 19 already downloaded
-File 20 already downloaded
-File 21 already downloaded
-File 22 already downloaded
-File 23 already downloaded
-File 24 already downloaded
-File 25 already downloaded


3. Зчитати завантажені текстові файли у фрейм
(детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не
повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

In [3]:
def data_in_frame(path):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    dataframes = []
    for filename in os.listdir(path):
        if not filename.startswith("vhi_id_"):
            continue
        index = filename.split('_')[2]
        filepath = os.path.join(path, filename)
        df = pd.read_csv(filepath, header=1, names=headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df['Region'] = index
        dataframes.append(df)
    main_df = pd.concat(dataframes, ignore_index=True).drop_duplicates().reset_index(drop=True)

    main_df['Year'] = main_df['Year'].str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')
    # main_df = main_df.drop(df.loc[df['Year'] == -1].index)
    main_df  = main_df[main_df['Year'].astype(str).str.strip() != '']
    main_df = main_df.drop(columns='empty')
    main_df['Year'] = main_df['Year'].astype(int)
    main_df['Week'] = main_df['Week'].astype(int)
    main_df['Region'] = main_df['Region'].astype(int)
    return main_df


data_frame = data_in_frame('C:/Users/ender/PycharmProjects/pythonProject/DataScienceAssigments/csv_prov')
print(data_frame)

       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      1982     1  0.059  258.24  51.11  48.78  49.95      10
1      1982     2  0.063  261.53  55.89  38.20  47.04      10
2      1982     3  0.063  263.45  57.30  32.69  44.99      10
3      1982     4  0.061  265.10  53.96  28.62  41.29      10
4      1982     5  0.058  266.42  46.87  28.57  37.72      10
...     ...   ...    ...     ...    ...    ...    ...     ...
54669  2024    48  0.135  278.17  55.23  11.86  33.55       9
54670  2024    49  0.133  277.08  57.71  10.86  34.29       9
54671  2024    50  0.130  276.49  59.45   8.68  34.07       9
54672  2024    51  0.128  276.45  62.53   5.55  34.04       9
54673  2024    52  0.129  276.48  66.13   3.71  34.92       9

[54650 rows x 8 columns]


4. Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [4]:
def convert_region_indices(dataframe, column="Region"):
    region_mapping = {
        24: 1, 25: 2, 5: 3, 6: 4, 27: 5, 23: 6, 26: 7, 7: 8,
        11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15,
        19: 16, 21: 17, 22: 18, 8: 19, 9: 20, 10: 21, 1: 22, 3: 23,
        2: 24, 4: 25, 12: 26, 20: 27
    }

    dataframe[column] = dataframe[column].map(region_mapping).fillna(dataframe[column]).astype(int)
    return dataframe


converted_datafr = convert_region_indices(data_frame)
print(converted_datafr)

       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      1982     1  0.059  258.24  51.11  48.78  49.95      21
1      1982     2  0.063  261.53  55.89  38.20  47.04      21
2      1982     3  0.063  263.45  57.30  32.69  44.99      21
3      1982     4  0.061  265.10  53.96  28.62  41.29      21
4      1982     5  0.058  266.42  46.87  28.57  37.72      21
...     ...   ...    ...     ...    ...    ...    ...     ...
54669  2024    48  0.135  278.17  55.23  11.86  33.55      20
54670  2024    49  0.133  277.08  57.71  10.86  34.29      20
54671  2024    50  0.130  276.49  59.45   8.68  34.07      20
54672  2024    51  0.128  276.45  62.53   5.55  34.04      20
54673  2024    52  0.129  276.48  66.13   3.71  34.92      20

[54650 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

In [5]:
def vhi_year(df, index, year):
    return df[(df["Region"] == index) & (df["Year"] == year)]['VHI']

print(vhi_year(converted_datafr, 2, 1996))

35690    34.95
35691    35.19
35692    35.38
35693    37.45
35694    40.18
35695    41.76
35696    42.01
35697    42.48
35698    43.23
35699    43.63
35700    43.96
35701    44.67
35702    44.57
35703    45.15
35704    41.82
35705    33.62
35706    29.71
35707    28.84
35708    32.86
35709    36.05
35710    37.55
35711    40.19
35712    41.37
35713    40.69
35714    42.60
35715    44.52
35716    46.55
35717    47.76
35718    49.18
35719    52.20
35720    51.83
35721    49.93
35722    48.68
35723    48.51
35724    49.53
35725    51.51
35726    54.73
35727    55.18
35728    54.61
35729    50.86
35730    47.32
35731    45.29
35732    45.59
35733    43.68
35734    38.90
35735    38.32
35736    39.39
35737    39.79
35738    38.98
35739    38.42
35740    40.29
35741    43.37
Name: VHI, dtype: float64


o Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;

In [6]:
def min_max(df, index,year):
    max_vhi = vhi_year(df, index, year).max()
    min_vhi = vhi_year(df, index, year).min()
    mean_vhi = vhi_year(df, index, year).mean()
    median_vhi = vhi_year(df, index, year).median()
    print(f'### Region: {index} ###\nMax vhi: {max_vhi}\nMin vhi: {min_vhi}\nMean vhi: {round(mean_vhi, 2)}\nMedian vhi: {round(median_vhi,2)}')

for x in range(1,29):
    min_max(converted_datafr, x, 2000)

### Region: 1 ###
Max vhi: 63.27
Min vhi: 11.25
Mean vhi: 40.98
Median vhi: 40.91
### Region: 2 ###
Max vhi: 68.85
Min vhi: 24.65
Mean vhi: 50.06
Median vhi: 50.36
### Region: 3 ###
Max vhi: 61.55
Min vhi: 17.77
Mean vhi: 44.0
Median vhi: 46.06
### Region: 4 ###
Max vhi: 69.82
Min vhi: 17.4
Mean vhi: 45.83
Median vhi: 43.62
### Region: 5 ###
Max vhi: nan
Min vhi: nan
Mean vhi: nan
Median vhi: nan
### Region: 6 ###
Max vhi: 55.16
Min vhi: 20.12
Mean vhi: 42.17
Median vhi: 45.94
### Region: 7 ###
Max vhi: nan
Min vhi: nan
Mean vhi: nan
Median vhi: nan
### Region: 8 ###
Max vhi: 60.12
Min vhi: 18.98
Mean vhi: 42.26
Median vhi: 45.98
### Region: 9 ###
Max vhi: 61.87
Min vhi: 10.6
Mean vhi: 39.76
Median vhi: 35.92
### Region: 10 ###
Max vhi: 60.69
Min vhi: 16.84
Mean vhi: 42.44
Median vhi: 41.74
### Region: 11 ###
Max vhi: 74.06
Min vhi: 22.92
Mean vhi: 46.33
Median vhi: 43.28
### Region: 12 ###
Max vhi: 69.96
Min vhi: 29.53
Mean vhi: 46.68
Median vhi: 46.53
### Region: 13 ###
Max vhi: 60.4

o Ряд VHI за вказаний діапазон років для вказаних областей;

In [7]:
def vhi_row_year_region(df, regions, start_year, end_year):
    for region in regions:
        print(f"Region {region}:")
        vhi = df[(df['Region'] == region) & (df['Year'].between(start_year, end_year))]
        if not vhi.empty:
            print(vhi)
        else:
            print(f"No data available for region {region} for year {year}")
    print("\n")

vhi_row_year_region(converted_datafr, [1,6,2], 2000, 2005)

Region 1:
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
33711  2000     1  0.023  260.25   8.46  39.97  24.22       1
33712  2000     2  0.023  259.38  10.27  45.13  27.70       1
33713  2000     3  0.024  259.61  14.76  46.60  30.68       1
33714  2000     4  0.027  260.21  19.52  45.57  32.55       1
33715  2000     5  0.030  260.59  22.24  47.21  34.73       1
...     ...   ...    ...     ...    ...    ...    ...     ...
33998  2005    48  0.105  272.76  66.36  18.15  42.25       1
33999  2005    49  0.091  269.76  67.13  24.48  45.80       1
34000  2005    50  0.081  267.22  68.19  27.23  47.71       1
34001  2005    51  0.072  265.02  66.67  27.93  47.30       1
34002  2005    52  0.064  262.92  58.78  31.31  45.04       1

[292 rows x 8 columns]
Region 6:
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
31524  2000     1  0.055  260.11  13.44  59.94  36.69       6
31525  2000     2  0.057  259.67  15.73  66.01  40.87       6
31526  2000     3  0.059  

o Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [8]:
def drought_stats(df, percent):
    total_regions = df['Region'].nunique()
    extreme_drought_data = {}
    for year in range(df['Year'].min(), df['Year'].max() + 1):
        affected_regions = df[(df['Year'] == year) & (df['VHI'] < 15)]
        unique_regions = affected_regions['Region'].unique()
        if len(unique_regions) > total_regions * (percent / 100):
            extreme_drought_data[year] = {
                "regions": list(unique_regions),
                "VHI_values": list(affected_regions['VHI'])
            }
    if extreme_drought_data:
        print(f"Years with extreme drought affecting more than {percent}% of the regions in Ukraine:")
        for year, data in extreme_drought_data.items():
            print(f"Year: {year}\nRegions: {data['regions']}\nVHI: {data['VHI_values']}\n")
    else:
        print(f"No years found where more than {percent}% of the regions experienced extreme drought.")

drought_stats(converted_datafr, 10)

Years with extreme drought affecting more than 10% of the regions in Ukraine:
Year: 2000
Regions: [9, 26, 22, 27, 1, 19]
VHI: [12.51, 10.6, 11.2, 12.32, 14.65, 14.89, 12.76, 7.81, 6.49, 6.58, 6.71, 7.56, 9.25, 10.94, 12.28, 14.64, 11.82, 10.81, 10.68, 12.3, 14.24, 13.14, 9.5, 8.14, 9.69, 11.2, 11.36, 12.77, 12.26, 11.28, 11.25, 11.38, 12.91, 14.2, 14.61, 11.33, 9.36, 9.45, 9.73, 11.45, 14.29]

Year: 2007
Regions: [13, 14, 25, 20]
VHI: [11.44, 7.78, 6.44, 6.12, 6.11, 5.94, 6.36, 7.61, 9.45, 11.7, 11.88, 9.21, 7.08, 5.9, 5.52, 5.85, 6.82, 8.86, 11.91, 14.98, 14.23, 13.79, 13.41, 13.28, 14.36, 12.41, 12.23, 12.99, 13.33, 12.88, 12.63, 12.96, 13.48, 14.05, 14.41, 14.81]

